In [ ]:
import numpy as np
import os
# success case 

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import os
import pickle
from tqdm import tqdm
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

@dataclass
class ImageOnlyConfig:
    """Configuration for image-only processing"""
    
    # ===== PATHS =====
    IMAGE_FOLDER: str = "success_traj_img"
    
    OUTPUT_PATH: str = "image_features.npz"
    PCA_MODEL_PATH: str = "image_pca_models.pkl"
    
    # ===== IMAGE PROCESSING =====
    RESNET_FEATURE_DIM: int = 512  # ResNet18 final layer per view
    VIEWS: List[str] = None
    
    # ===== PCA COMPRESSION =====
    COMPRESSED_DIM: int = 64  # Final compressed dimension per view
    TOTAL_COMPRESSED_DIM: int = 192  # 64 * 3 views
    
    # ===== MODEL =====
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"
    BATCH_SIZE: int = 32
    
    def __post_init__(self):
        if self.VIEWS is None:
            self.VIEWS = ["front", "top", "wrist"]
        
        print(f"Image-Only Processor Config")
        print(f"Views: {self.VIEWS}")
        print(f"ResNet Features: {self.RESNET_FEATURE_DIM} per view")
        print(f"Compressed Features: {self.COMPRESSED_DIM} per view")
        print(f"Total Compressed: {self.TOTAL_COMPRESSED_DIM}")
        print(f"Device: {self.DEVICE}")

class EpisodeProcessor:
    def __init__(self, config: ImageOnlyConfig):
        self.config = config
        self.device = torch.device(config.DEVICE)

        # ResNet18 feature extractor
        self.model = models.resnet18(pretrained=True)
        self.model = nn.Sequential(*list(self.model.children())[:-1])
        self.model = self.model.to(self.device)
        self.model.eval()

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])
        ])

        self.pca_models = {}
        print(f"[INFO] ResNet18 feature extractor initialized on {self.device}")

    def extract_features(self, image_path: str) -> np.ndarray:
        try:
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
            with torch.no_grad():
                features = self.model(image_tensor).view(1, -1)
            return features.cpu().numpy().flatten()
        except Exception as e:
            print(f"[WARN] Failed to process {image_path}: {e}")
            return np.zeros(self.config.RESNET_FEATURE_DIM)

    def process_episode(self, episode_dir: str) -> Dict[str, np.ndarray]:
        """Process a single episode directory"""
        print(f"[INFO] Processing episode: {episode_dir}")
        
        # Load robot state
        state_path = os.path.join(episode_dir, "robot_state.npz")
        if not os.path.exists(state_path):
            raise FileNotFoundError(f"No robot_state.npz found in {episode_dir}")
        state_data = np.load(state_path)
        state_key = list(state_data.keys())[0]  
        robot_states = state_data[state_key]
        print(f"[INFO] Robot state shape: {robot_states.shape}")

        # Build timestep list
        front_dir = os.path.join(episode_dir, "front_view")
        timesteps = sorted([
            int(f.split('_')[-1].replace('.png', ''))
            for f in os.listdir(front_dir) if f.endswith('.png')
        ])

        features = []
        for i, ts in enumerate(timesteps):
            view_feats = []
            for view in self.config.VIEWS:
                img_path = os.path.join(episode_dir, f"{view}_view", f"{view}_view_{ts}.png")
                feat = self.extract_features(img_path)
                view_feats.append(feat)

            combined_img_feat = np.concatenate(view_feats)  # [1536]
            features.append(np.concatenate([combined_img_feat, robot_states[i]]))

        return {"observation": np.vstack(features)}

    def fit_pca(self, all_episode_features: List[np.ndarray]):
        """Fit PCA per view across all episodes"""
        print("[INFO] Fitting PCA models...")

        total_img_dim = len(self.config.VIEWS) * self.config.RESNET_FEATURE_DIM
        sample_feat = all_episode_features[0]
        state_dim = sample_feat.shape[1] - total_img_dim
        print(f"[INFO] Detected state_dim = {state_dim}")

        view_features = {view: [] for view in self.config.VIEWS}

        for episode_feat in all_episode_features:
            img_feats = episode_feat[:, :-state_dim]
            for i, view in enumerate(self.config.VIEWS):
                start, end = i * self.config.RESNET_FEATURE_DIM, (i + 1) * self.config.RESNET_FEATURE_DIM
                view_features[view].append(img_feats[:, start:end])

        for view in self.config.VIEWS:
            X = np.vstack(view_features[view])  # (N*T, 512)
            pca = PCA(n_components=self.config.COMPRESSED_DIM)
            pca.fit(X)
            self.pca_models[view] = pca
            print(f"[INFO] {view} view PCA variance explained: {pca.explained_variance_ratio_.sum():.3f}")

    def compress_episode(self, episode_dict: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
        """Apply PCA compression to images, concat state as-is"""
        obs = episode_dict["observation"]
        total_img_dim = len(self.config.VIEWS) * self.config.RESNET_FEATURE_DIM
        state_dim = obs.shape[1] - total_img_dim

        img_feats, state_feats = obs[:, :-state_dim], obs[:, -state_dim:]

        compressed_features = []
        for row in img_feats:
            comp_views = []
            for i, view in enumerate(self.config.VIEWS):
                start, end = i*self.config.RESNET_FEATURE_DIM, (i+1)*self.config.RESNET_FEATURE_DIM
                view_feat = row[start:end].reshape(1, -1)
                comp_views.append(self.pca_models[view].transform(view_feat).flatten())
            compressed_features.append(np.concatenate(comp_views))

        compressed_features = np.vstack(compressed_features)
        final_obs = np.hstack([compressed_features, state_feats])  # PCA된 이미지 + 원본 state
        return {"observation": final_obs}

import os
import numpy as np
from tqdm import tqdm

if __name__ == "__main__":
    config = ImageOnlyConfig(
        IMAGE_FOLDER="/AILAB-summer-school-2025/success_data_raw", # image path 
        OUTPUT_PATH="success_data_preprocessing",
        PCA_MODEL_PATH="pca_models.pkl",
        COMPRESSED_DIM=64
    )
    processor = EpisodeProcessor(config)

    # Step 1. 전체 episode 폴더 탐색
    episode_dirs = [
        os.path.join(config.IMAGE_FOLDER, d)
        for d in os.listdir(config.IMAGE_FOLDER)
        if os.path.isdir(os.path.join(config.IMAGE_FOLDER, d))
        and ("success_" in d or "fail_" in d)
    ]
    print(f"[INFO] Found {len(episode_dirs)} episodes.")

    # Step 2. 각 episode feature 추출
    all_episode_features = []
    raw_episode_dicts = {}
    for epi_dir in tqdm(episode_dirs, desc="Processing episodes"):
        try:
            epi_name = os.path.basename(epi_dir)
            episode_dict = processor.process_episode(epi_dir)  # raw features + state
            raw_episode_dicts[epi_name] = episode_dict
            all_episode_features.append(episode_dict["observation"])
        except Exception as e:
            print(f"[WARN] Skipping {epi_dir}: {e}")

    # Step 3. PCA 학습 (view별)
    processor.fit_pca(all_episode_features)

    # Step 4. PCA 압축 적용 및 저장
    output_dir = "compressed_episodes"
    os.makedirs(output_dir, exist_ok=True)

    for epi_name, epi_dict in raw_episode_dicts.items():
        compressed_dict = processor.compress_episode(epi_dict)

        save_path = os.path.join(output_dir, f"{epi_name}.npz")
        np.savez_compressed(save_path, **compressed_dict)
        
        print(f"[INFO] Saved compressed episode: {save_path}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Image-Only Processor Config
Views: ['front', 'top', 'wrist']
ResNet Features: 512 per view
Compressed Features: 64 per view
Total Compressed: 192
Device: cuda
[INFO] ResNet18 feature extractor initialized on cuda
[INFO] Found 5 episodes.


Processing episodes: 100%|██████████| 5/5 [00:00<00:00, 25795.23it/s]

[INFO] Processing episode: /AILAB-summer-school-2025/fail_data_raw/fail_case1_missingGrasp
[WARN] Skipping /AILAB-summer-school-2025/fail_data_raw/fail_case1_missingGrasp: No robot_state.npz found in /AILAB-summer-school-2025/fail_data_raw/fail_case1_missingGrasp
[INFO] Processing episode: /AILAB-summer-school-2025/fail_data_raw/fail_case4_release
[WARN] Skipping /AILAB-summer-school-2025/fail_data_raw/fail_case4_release: No robot_state.npz found in /AILAB-summer-school-2025/fail_data_raw/fail_case4_release
[INFO] Processing episode: /AILAB-summer-school-2025/fail_data_raw/fail_case2_outofcontrolPregrasp
[WARN] Skipping /AILAB-summer-school-2025/fail_data_raw/fail_case2_outofcontrolPregrasp: No robot_state.npz found in /AILAB-summer-school-2025/fail_data_raw/fail_case2_outofcontrolPregrasp
[INFO] Processing episode: /AILAB-summer-school-2025/fail_data_raw/fail_case3_outofcontrolMovetobin
[WARN] Skipping /AILAB-summer-school-2025/fail_data_raw/fail_case3_outofcontrolMovetobin: No robot_

IndexError: list index out of range

In [ ]:
import numpy as np
import os
# success case 

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import os
import pickle
from tqdm import tqdm
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

@dataclass
class ImageOnlyConfig:
    """Configuration for image-only processing"""
    
    # ===== PATHS =====
    IMAGE_FOLDER: str = "success_traj_img"
    
    OUTPUT_PATH: str = "image_features.npz"
    PCA_MODEL_PATH: str = "image_pca_models.pkl"
    
    # ===== IMAGE PROCESSING =====
    RESNET_FEATURE_DIM: int = 512  # ResNet18 final layer per view
    VIEWS: List[str] = None
    
    # ===== PCA COMPRESSION =====
    COMPRESSED_DIM: int = 64  # Final compressed dimension per view
    TOTAL_COMPRESSED_DIM: int = 192  # 64 * 3 views
    
    # ===== MODEL =====
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"
    BATCH_SIZE: int = 32
    
    def __post_init__(self):
        if self.VIEWS is None:
            self.VIEWS = ["front", "top", "wrist"]
        
        print(f"Image-Only Processor Config")
        print(f"Views: {self.VIEWS}")
        print(f"ResNet Features: {self.RESNET_FEATURE_DIM} per view")
        print(f"Compressed Features: {self.COMPRESSED_DIM} per view")
        print(f"Total Compressed: {self.TOTAL_COMPRESSED_DIM}")
        print(f"Device: {self.DEVICE}")

class EpisodeProcessor:
    def __init__(self, config: ImageOnlyConfig):
        self.config = config
        self.device = torch.device(config.DEVICE)

        # ResNet18 feature extractor
        self.model = models.resnet18(pretrained=True)
        self.model = nn.Sequential(*list(self.model.children())[:-1])
        self.model = self.model.to(self.device)
        self.model.eval()

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])
        ])

        self.pca_models = {}
        print(f"[INFO] ResNet18 feature extractor initialized on {self.device}")

    def extract_features(self, image_path: str) -> np.ndarray:
        try:
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
            with torch.no_grad():
                features = self.model(image_tensor).view(1, -1)
            return features.cpu().numpy().flatten()
        except Exception as e:
            print(f"[WARN] Failed to process {image_path}: {e}")
            return np.zeros(self.config.RESNET_FEATURE_DIM)

    def process_episode(self, episode_dir: str) -> Dict[str, np.ndarray]:
        """Process a single episode directory"""
        print(f"[INFO] Processing episode: {episode_dir}")
        
        # Load robot state
        state_path = os.path.join(episode_dir, "robot_state.npz")
        if not os.path.exists(state_path):
            raise FileNotFoundError(f"No robot_state.npz found in {episode_dir}")
        state_data = np.load(state_path)
        state_key = list(state_data.keys())[0]  
        robot_states = state_data[state_key]
        print(f"[INFO] Robot state shape: {robot_states.shape}")

        # Build timestep list
        front_dir = os.path.join(episode_dir, "front_view")
        timesteps = sorted([
            int(f.split('_')[-1].replace('.png', ''))
            for f in os.listdir(front_dir) if f.endswith('.png')
        ])

        features = []
        for i, ts in enumerate(timesteps):
            view_feats = []
            for view in self.config.VIEWS:
                img_path = os.path.join(episode_dir, f"{view}_view", f"{view}_view_{ts}.png")
                feat = self.extract_features(img_path)
                view_feats.append(feat)

            combined_img_feat = np.concatenate(view_feats)  # [1536]
            features.append(np.concatenate([combined_img_feat, robot_states[i]]))

        return {"observation": np.vstack(features)}

    def fit_pca(self, all_episode_features: List[np.ndarray]):
        """Fit PCA per view across all episodes"""
        print("[INFO] Fitting PCA models...")

        total_img_dim = len(self.config.VIEWS) * self.config.RESNET_FEATURE_DIM
        sample_feat = all_episode_features[0]
        state_dim = sample_feat.shape[1] - total_img_dim
        print(f"[INFO] Detected state_dim = {state_dim}")

        view_features = {view: [] for view in self.config.VIEWS}

        for episode_feat in all_episode_features:
            img_feats = episode_feat[:, :-state_dim]
            for i, view in enumerate(self.config.VIEWS):
                start, end = i * self.config.RESNET_FEATURE_DIM, (i + 1) * self.config.RESNET_FEATURE_DIM
                view_features[view].append(img_feats[:, start:end])

        for view in self.config.VIEWS:
            X = np.vstack(view_features[view])  # (N*T, 512)
            pca = PCA(n_components=self.config.COMPRESSED_DIM)
            pca.fit(X)
            self.pca_models[view] = pca
            print(f"[INFO] {view} view PCA variance explained: {pca.explained_variance_ratio_.sum():.3f}")

    def compress_episode(self, episode_dict: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
        """Apply PCA compression to images, concat state as-is"""
        obs = episode_dict["observation"]
        total_img_dim = len(self.config.VIEWS) * self.config.RESNET_FEATURE_DIM
        state_dim = obs.shape[1] - total_img_dim

        img_feats, state_feats = obs[:, :-state_dim], obs[:, -state_dim:]

        compressed_features = []
        for row in img_feats:
            comp_views = []
            for i, view in enumerate(self.config.VIEWS):
                start, end = i*self.config.RESNET_FEATURE_DIM, (i+1)*self.config.RESNET_FEATURE_DIM
                view_feat = row[start:end].reshape(1, -1)
                comp_views.append(self.pca_models[view].transform(view_feat).flatten())
            compressed_features.append(np.concatenate(comp_views))

        compressed_features = np.vstack(compressed_features)
        final_obs = np.hstack([compressed_features, state_feats])  # PCA된 이미지 + 원본 state
        return {"observation": final_obs}

import os
import numpy as np
from tqdm import tqdm

def format_episode_name(epi_dir: str) -> str:
    parts = epi_dir.split(os.sep)
    base = parts[-1]       
    parent = parts[-2] if len(parts) >= 2 else ""

    if parent.startswith("fail_case"):
        failtype_num = ''.join([c for c in parent if c.isdigit()])
        new_base = base.replace(f"fail{failtype_num}_", f"failtype{failtype_num}_")
        return f"{new_base}"
    else:
        return base  

if __name__ == "__main__":
    config = ImageOnlyConfig(
        IMAGE_FOLDER="/AILAB-summer-school-2025/fail_data_raw", # image path 
        OUTPUT_PATH="final_episode_dicts.npz",
        PCA_MODEL_PATH="/AILAB-summer-school-2025/success_data_preprocessing/pca_weight/image_pca_models.pkl",
        COMPRESSED_DIM=64)

    processor = EpisodeProcessor(config)

    # Step 1. 전체 episode 폴더 탐색 (success/fail 구분)
    episode_dirs = []
    for root in os.listdir(config.IMAGE_FOLDER):
        root_path = os.path.join(config.IMAGE_FOLDER, root)
        if not os.path.isdir(root_path):
            continue

        if root.startswith("success_"):  
            episode_dirs.append(root_path)
        elif root.startswith("fail_case"):
            for sub in os.listdir(root_path):
                sub_path = os.path.join(root_path, sub)
                if os.path.isdir(sub_path) and sub.startswith("fail"):
                    episode_dirs.append(sub_path)

    print(f"[INFO] Found {len(episode_dirs)} episodes.")

    # Step 2. 각 episode feature 추출
    all_episode_features = []
    raw_episode_dicts = {}
    for epi_dir in tqdm(episode_dirs, desc="Processing episodes"):
        try:
            epi_name = format_episode_name(epi_dir)
            episode_dict = processor.process_episode(epi_dir)
            raw_episode_dicts[epi_name] = episode_dict
            all_episode_features.append(episode_dict["observation"])
        except Exception as e:
            print(f"[WARN] Skipping {epi_dir}: {e}")

    # Step 3. PCA 학습 (view별)
    processor.fit_pca(all_episode_features)

    # Step 4. PCA 압축 적용 및 저장
    output_dir = "compressed_episodes"
    os.makedirs(output_dir, exist_ok=True)

    for epi_name, epi_dict in raw_episode_dicts.items():
        compressed_dict = processor.compress_episode(epi_dict)
        epi_name = epi_name.replace("/", "_")
        save_path = os.path.join(output_dir, f"{epi_name}.npz")
        np.savez_compressed(save_path, **compressed_dict)
        print(f"[INFO] Saved compressed episode: {save_path}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Image-Only Processor Config
Views: ['front', 'top', 'wrist']
ResNet Features: 512 per view
Compressed Features: 64 per view
Total Compressed: 192
Device: cuda
[INFO] ResNet18 feature extractor initialized on cuda
[INFO] Found 22 episodes.


Processing episodes:   0%|          | 0/22 [00:00<?, ?it/s]

[INFO] Processing episode: /AILAB-summer-school-2025/fail_data_raw/fail_case1_missingGrasp/fail1_episode5_step349
[INFO] Robot state shape: (70, 7)


Processing episodes:   5%|▍         | 1/22 [00:00<00:10,  1.99it/s]

...

[INFO] Processing episode: /AILAB-summer-school-2025/fail_data_raw/fail_case5_light/fail5_episode5_step349
[INFO] Robot state shape: (70, 7)


Processing episodes:  95%|█████████▌| 21/22 [00:12<00:00,  1.57it/s]

[INFO] Processing episode: /AILAB-summer-school-2025/fail_data_raw/fail_case5_light/fail5_episode4_step349
[INFO] Robot state shape: (70, 7)


Processing episodes: 100%|██████████| 22/22 [00:13<00:00,  1.60it/s]


[INFO] Fitting PCA models...
[INFO] Detected state_dim = 7
[INFO] front view PCA variance explained: 0.938
[INFO] top view PCA variance explained: 0.944
[INFO] wrist view PCA variance explained: 0.935
[INFO] Saved compressed episode: compressed_episodes/fail_case1_missingGrasp_failtype1_episode5_step349.npz
[INFO] Saved compressed episode: compressed_episodes/fail_case1_missingGrasp_failtype1_episode3_step349.npz
[INFO] Saved compressed episode: compressed_episodes/fail_case1_missingGrasp_failtype1_episode2_step349.npz
[INFO] Saved compressed episode: compressed_episodes/fail_case1_missingGrasp_failtype1_episode4_step349.npz
[INFO] Saved compressed episode: compressed_episodes/fail_case1_missingGrasp_failtype1_episode1_step349.npz
[INFO] Saved compressed episode: compressed_episodes/fail_case4_release_failtype4_episode2_step349.npz
[INFO] Saved compressed episode: compressed_episodes/fail_case4_release_failtype4_episode4_step349.npz
[INFO] Saved compressed episode: compressed_episodes/

In [ ]:
# ------------------------------------------------------------
# Key: img
#   Shape: (97, 16)
#   Dtype: float32
#   Sample (first element):
# [-0.5854379   1.6493505  -1.0049931  -0.21947424 -0.70256466 -0.39046937
#  -0.4245706  -0.36956656  0.24001157 -0.14053325]
# ------------------------------------------------------------
# Key: robot_state
#   Shape: (97, 9)
#   Dtype: float32
#   Sample (first element):
# [ 4.6333355e-01  5.1339157e-08  3.8548785e-01  8.6034834e-03
#   9.2161107e-01  2.0462854e-02  3.8747975e-01 -3.5621226e-05
#   0.0000000e+00  3.0250198e-01]
# ------------------------------------------------------------


import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np

# ----- Image feature extractor -----
def get_image_feature(image_path, model, transform, device="cpu"):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch
    with torch.no_grad():
        features = model(image)
    return features.squeeze().cpu().numpy()

# ----- Main processing function -----
def process_success_traj(base_dir, output_dir, view="top"):
    """
    base_dir: success_traj root directory (/AILAB-summer-school-2025/success_traj/)
    output_dir: where to save merged dicts
    view: "front", "top", or "wrist"
    """

    os.makedirs(output_dir, exist_ok=True)

    # Pretrained ResNet18 -> 16D feature
    device = "cuda" if torch.cuda.is_available() else "cpu"
    resnet18 = models.resnet18(pretrained=True)
    num_ftrs = resnet18.fc.in_features
    resnet18.fc = nn.Linear(num_ftrs, 16)
    resnet18 = resnet18.to(device)
    resnet18.eval()

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    # Traverse all traj folders
    for traj_folder in sorted(os.listdir(base_dir)):
        traj_path = os.path.join(base_dir, traj_folder)
        if not os.path.isdir(traj_path) or not traj_folder.startswith("simulation_traj_"):
            continue

        traj_num = traj_folder.split("_")[2]
        dict_name = f"success_traj_{traj_num}_{view}"

        # Collect image features
        img_features = []
        for filename in sorted(os.listdir(traj_path)):
            if filename.startswith(f"{view}_view") and filename.endswith(".png"):
                image_path = os.path.join(traj_path, filename)
                feat = get_image_feature(image_path, resnet18, transform, device)
                img_features.append(feat)

        img_features = np.stack(img_features, axis=0)  # shape: [num_images, feature_dim]

        # Collect robot states from all timestep npz
        all_states = []
        for file in sorted(os.listdir(traj_path)):
            if file.endswith(".npz") and file.startswith("states_"):
                file_path = os.path.join(traj_path, file)
                data = np.load(file_path)
                for key in data.files:
                    all_states.append(data[key])

        robot_states = np.concatenate(all_states, axis=0)

        # Build dict
        traj_dict = {
            "img": img_features,
            "robot_state": robot_states
        }

        # Save npz
        save_path = os.path.join(output_dir, f"{dict_name}.npz")
        np.savez(save_path, **traj_dict)

        print(f"[Saved] {save_path}: img {img_features.shape}, state {robot_states.shape}")


if __name__ == "__main__":
    base_dir = "/AILAB-summer-school-2025/success_traj/"
    output_dir = "/AILAB-summer-school-2025/success_traj_comp/"
    process_success_traj(base_dir, output_dir, view="wrist")  # front/top/wrist 중 선택

In [ ]:
import numpy as np

def inspect_npz(file_path):
    """
    Inspect contents of a .npz file: keys, shapes, and dtypes
    """
    data = np.load(file_path)
    print(f"Inspecting {file_path}")
    print("-" * 60)
    for key in data.files:
        print(f"Key: {key}")
        print(f"  Shape: {data[key].shape}")
        print(f"  Dtype: {data[key].dtype}")
        print(f"  Sample (first element):\n{data[key].flatten()[:10]}")
        print("-" * 60)

if __name__ == "__main__":
    file_path = "/AILAB-summer-school-2025/success_traj/success_traj_comp_front/success_traj_2_front.npz"
    inspect_npz(file_path)


In [ ]:
# success case 

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import os
import pickle
from tqdm import tqdm
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

@dataclass
class ImageOnlyConfig:
    """Configuration for image-only processing"""
    
    # ===== PATHS =====
    IMAGE_FOLDER: str = "success_traj_img"
    
    OUTPUT_PATH: str = "image_features.npz"
    PCA_MODEL_PATH: str = "image_pca_models.pkl"
    
    # ===== IMAGE PROCESSING =====
    RESNET_FEATURE_DIM: int = 512  # ResNet18 final layer per view
    VIEWS: List[str] = None
    
    # ===== PCA COMPRESSION =====
    COMPRESSED_DIM: int = 64  # Final compressed dimension per view
    TOTAL_COMPRESSED_DIM: int = 192  # 64 * 3 views
    
    # ===== MODEL =====
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"
    BATCH_SIZE: int = 32
    
    def __post_init__(self):
        if self.VIEWS is None:
            self.VIEWS = ["front", "top", "wrist"]
        
        print(f"Image-Only Processor Config")
        print(f"Views: {self.VIEWS}")
        print(f"ResNet Features: {self.RESNET_FEATURE_DIM} per view")
        print(f"Compressed Features: {self.COMPRESSED_DIM} per view")
        print(f"Total Compressed: {self.TOTAL_COMPRESSED_DIM}")
        print(f"Device: {self.DEVICE}")

class ImageOnlyProcessor:
    """Process only images to create latent vectors"""
    
    def __init__(self, config: ImageOnlyConfig):
        self.config = config
        self.device = torch.device(config.DEVICE)

        # Initialize ResNet18
        self.model = models.resnet18(pretrained=True)
        self.model = nn.Sequential(*list(self.model.children())[:-1])  # Remove classifier
        self.model = self.model.to(self.device)
        self.model.eval()
        
        # Image preprocessing
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        # Storage
        self.image_index = {}
        self.pca_models = {}
        
        print(f"ResNet18 feature extractor initialized")
    
    def parse_filename(self, filename: str) -> Optional[Tuple[str, str, int]]:
        """Parse image filename: traj_key, view, timestep"""
        name = filename.replace('.png', '')
        parts = name.split('_')
        
        try:
            # Find view
            view = None
            view_idx = -1
            for i, part in enumerate(parts):
                if part in self.config.VIEWS:
                    view = part
                    view_idx = i
                    break
            
            if view is None:
                return None
            
            # Extract trajectory key and timestep
            traj_key = '_'.join(parts[:view_idx])
            timestep = int(parts[-1])
            
            return traj_key, view, timestep
            
        except (ValueError, IndexError):
            return None
    
    def build_image_index(self):
        print(f"Building image index from: {self.config.IMAGE_FOLDER}")
        image_index = defaultdict(lambda: defaultdict(dict))
        total_images, parsed_images = 0, 0

        for root, _, files in os.walk(self.config.IMAGE_FOLDER):
            for filename in files:
                if not filename.endswith('.png'):
                    continue
                total_images += 1
                parse_result = self.parse_filename(filename)
                if parse_result:
                    traj_key, view, timestep = parse_result
                    image_path = os.path.join(root, filename)
                    image_index[traj_key][timestep][view] = image_path
                    parsed_images += 1

        complete_triplets = sum(
            len(image_index[traj][ts]) == len(self.config.VIEWS)
            for traj in image_index for ts in image_index[traj]
        )

        print(f"Total images: {total_images}, Parsed: {parsed_images}, Complete triplets: {complete_triplets}")
        self.image_index = dict(image_index)
        return complete_triplets
    
    def extract_features(self, image_path: str) -> np.ndarray:
        """Extract ResNet18 features from single image"""
        try:
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
            
            with torch.no_grad():
                features = self.model(image_tensor)
                features = features.view(features.size(0), -1)
            
            return features.cpu().numpy().flatten()
        
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            return np.zeros(self.config.RESNET_FEATURE_DIM)
    
    def extract_all_image_features(self) -> Dict[str, Dict[int, np.ndarray]]:
        """Extract features for all complete image triplets"""
        print("Extracting multiview image features...")
        
        features_dict = {}
        total_processed = 0
        
        for traj_key in tqdm(self.image_index, desc="Processing trajectories"):
            features_dict[traj_key] = {}
            
            for timestep in self.image_index[traj_key]:
                # Check if all views available
                available_views = set(self.image_index[traj_key][timestep].keys())
                required_views = set(self.config.VIEWS)
                
                if available_views == required_views:
                    # Extract features from all 3 views
                    view_features = []
                    
                    for view in self.config.VIEWS:
                        image_path = self.image_index[traj_key][timestep][view]
                        features = self.extract_features(image_path)
                        view_features.append(features)
                    
                    # Concatenate all view features
                    combined_features = np.concatenate(view_features)  # [1536,]
                    features_dict[traj_key][timestep] = combined_features
                    total_processed += 1
        
        print(f"Extracted features for {total_processed} complete image triplets")
        return features_dict
    
    def fit_pca_models(self, features_dict: Dict) -> Dict[str, PCA]:
        """Fit PCA for each view separately"""
        print("Fitting PCA compression models...")
        
        # Collect features by view
        view_features = {view: [] for view in self.config.VIEWS}
        
        for traj_key in features_dict:
            for timestep in features_dict[traj_key]:
                combined_features = features_dict[traj_key][timestep]
                
                # Split by view
                for i, view in enumerate(self.config.VIEWS):
                    start_idx = i * self.config.RESNET_FEATURE_DIM
                    end_idx = (i + 1) * self.config.RESNET_FEATURE_DIM
                    view_feature = combined_features[start_idx:end_idx]
                    view_features[view].append(view_feature)
        
        # Fit PCA for each view
        pca_models = {}
        for view in self.config.VIEWS:
            if view_features[view]:
                features_array = np.array(view_features[view])
                
                pca = PCA(n_components=self.config.COMPRESSED_DIM)
                pca.fit(features_array)
                
                explained_var = pca.explained_variance_ratio_.sum()
                print(f"  {view} view: {explained_var:.3f} variance explained")
                
                pca_models[view] = pca
        
        self.pca_models = pca_models
        return pca_models
    
    def compress_all_features(self, features_dict: Dict) -> Dict[str, Dict[int, np.ndarray]]:
        """Apply PCA compression to all features"""
        print("Compressing features with PCA...")
        
        compressed_dict = {}
        
        for traj_key in tqdm(features_dict, desc="Compressing"):
            compressed_dict[traj_key] = {}
            
            for timestep in features_dict[traj_key]:
                combined_features = features_dict[traj_key][timestep]
                
                # Compress each view separately
                compressed_views = []
                for i, view in enumerate(self.config.VIEWS):
                    start_idx = i * self.config.RESNET_FEATURE_DIM
                    end_idx = (i + 1) * self.config.RESNET_FEATURE_DIM
                    view_feature = combined_features[start_idx:end_idx]
                    
                    if view in self.pca_models:
                        compressed_feature = self.pca_models[view].transform([view_feature])
                        compressed_views.append(compressed_feature.flatten())
                    else:
                        compressed_views.append(np.zeros(self.config.COMPRESSED_DIM))
                
                # Combine compressed features from all views
                final_compressed = np.concatenate(compressed_views)  # [192,]
                compressed_dict[traj_key][timestep] = final_compressed
        
        return compressed_dict
    
    def save_image_features(self, compressed_features: Dict):
        """Save image features only"""
        print(f"Saving image features to: {self.config.OUTPUT_PATH}")
        
        # Convert to arrays with metadata
        feature_list = []
        metadata_list = []
        
        for traj_key in compressed_features:
            for timestep in compressed_features[traj_key]:
                feature_vector = compressed_features[traj_key][timestep]
                feature_list.append(feature_vector)
                
                metadata_list.append({
                    'traj_key': traj_key,
                    'timestep': timestep,
                    'feature_dim': len(feature_vector)
                })
        
        feature_array = np.array(feature_list)
        
        # Save features
        np.savez_compressed(
            self.config.OUTPUT_PATH,
            features=feature_array,
            metadata=metadata_list,
            config=self.config.__dict__
        )
        
        # Save PCA models
        with open(self.config.PCA_MODEL_PATH, 'wb') as f:
            pickle.dump(self.pca_models, f)
        
        print(f"Image features saved:")
        print(f"  Features: {self.config.OUTPUT_PATH}")
        print(f"  PCA models: {self.config.PCA_MODEL_PATH}")
        print(f"  Total features: {len(feature_array)}")
        print(f"  Feature dimension: {feature_array.shape[1]}")
        print(f"  File size: {os.path.getsize(self.config.OUTPUT_PATH)/1024/1024:.1f} MB")

def process_images_only(config: ImageOnlyConfig = None) -> str:
    """
    Main function to process images only
    
    Returns:
        Path to generated image features file
    """
    if config is None:
        config = ImageOnlyConfig()
    
    print("=" * 60)
    print("Image-Only Processing Pipeline")
    print("Extracting latent vectors from multiview images")
    print("=" * 60)
    
    try:
        # Initialize processor
        processor = ImageOnlyProcessor(config)
        
        # Step 1: Build image index
        print("\n1. Building image index...")
        complete_count = processor.build_image_index()
        
        if complete_count == 0:
            raise ValueError("No complete image triplets found!")
        
        # Step 2: Extract raw features
        print("\n2. Extracting ResNet18 features...")
        features_dict = processor.extract_all_image_features()
        
        # Step 3: Fit PCA
        print("\n3. Fitting PCA compression...")
        processor.fit_pca_models(features_dict)
        
        # Step 4: Compress features
        print("\n4. Compressing features...")
        compressed_features = processor.compress_all_features(features_dict)
        
        # Step 5: Save results
        print("\n5. Saving image features...")
        processor.save_image_features(compressed_features)
        
        print("\n" + "=" * 60)
        print("Image-Only Processing Completed Successfully!")
        print("=" * 60)
        print(f"✅ Generated: {config.OUTPUT_PATH}")
        print(f"✅ Feature dimension: {config.TOTAL_COMPRESSED_DIM}")
        print(f"✅ Views processed: {config.VIEWS}")
        
        return config.OUTPUT_PATH
        
    except Exception as e:
        print(f"\n Processing failed: {e}")
        raise e

if __name__ == "__main__":
    config = ImageOnlyConfig()
    output_path = process_images_only(config)
    print(f"\nImage latent vectors ready: {output_path}")

In [ ]:
# failure case

import os, pickle, numpy as np
import torch, torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
from sklearn.decomposition import PCA
from collections import defaultdict

class MultiViewConfig:
    IMAGE_ROOT: str = "/AILAB-summer-school-2025/failure_case/failcase2"
    OUTPUT_ROOT: str = "/AILAB-summer-school-2025/failure_case_comp/failcase2"
    VIEWS: dict = {"front_view": "front", "top_view": "top", "wrist_view": "wrist"}
    RESNET_FEATURE_DIM: int = 512
    COMPRESSED_DIM: int = 64
    TOTAL_COMPRESSED_DIM: int = 64 * 3
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

class MultiViewProcessor:
    def __init__(self, config: MultiViewConfig):
        self.config = config
        self.device = torch.device(config.DEVICE)
        self.model = models.resnet18(pretrained=True)
        self.model = nn.Sequential(*list(self.model.children())[:-1])
        self.model.to(self.device).eval()
        self.transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406],
                                 std=[0.229,0.224,0.225])
        ])
        self.pca_models = {}
        print("Initialized ResNet18 for multiview")

    def extract_feature(self, image_path: str) -> np.ndarray:
        try:
            img = Image.open(image_path).convert("RGB")
            x = self.transform(img).unsqueeze(0).to(self.device)
            with torch.no_grad():
                feat = self.model(x).view(1,-1)
            return feat.cpu().numpy().flatten()
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            return np.zeros(self.config.RESNET_FEATURE_DIM)

    def parse_filename(self, filename: str):
        if not filename.endswith(".png"):
            return None
        name = filename.replace(".png", "")
        # view 매칭
        view = None
        for candidate in ["front_view", "top_view", "wrist_view"]:
            if candidate in name:
                view = candidate.replace("_view", "")
                break
        if view is None:
            return None
        try:
            timestep = int(name.split("_")[-1])
        except ValueError:
            return None
        traj_key = name.rsplit(f"_{view}_", 1)[0]
        return traj_key, view, timestep

    def fit_pca_models(self, feats: np.ndarray):
        """fit PCA per view from stacked feats"""
        pca_models, reduced_views = {}, []
        for i, view in enumerate(["front","top","wrist"]):
            view_data = feats[:, i*512:(i+1)*512]
            pca = PCA(n_components=self.config.COMPRESSED_DIM)
            pca.fit(view_data)
            explained = pca.explained_variance_ratio_.sum()
            print(f"  PCA fitted for {view}: explains {explained:.3f}")
            pca_models[view] = pca
        self.pca_models = pca_models
        return pca_models

    def process_traj(self, traj_dir):
        print(f"\nProcessing {traj_dir}")

        traj_name = os.path.basename(traj_dir)
        # output trajectory dir
        out_traj_dir = os.path.join(self.config.OUTPUT_ROOT, traj_name)
        img_out = os.path.join(out_traj_dir, "img")
        state_out = os.path.join(out_traj_dir, "robot_state")
        os.makedirs(img_out, exist_ok=True)
        os.makedirs(state_out, exist_ok=True)

        image_index = defaultdict(dict)
        for fname in os.listdir(traj_dir):
            res = self.parse_filename(fname)
            if res:
                traj_key, view, ts = res
                image_index[ts][view] = os.path.join(traj_dir, fname)

        timesteps = [t for t in image_index if len(image_index[t]) == 3]
        if not timesteps:
            print("  No complete triplets found")
            return
        timesteps = sorted(timesteps)

        # 이미지 feature 추출
        feats = []
        for t in timesteps:
            view_feats = []
            for view in ["front","top","wrist"]:
                feat = self.extract_feature(image_index[t][view])
                view_feats.append(feat)
            feats.append(np.concatenate(view_feats))
        feats = np.array(feats)

        # PCA transform (fit if first traj)
        if not self.pca_models:
            print("  Fitting PCA models...")
            self.fit_pca_models(feats)

        reduced_views = []
        for i, view in enumerate(["front","top","wrist"]):
            view_data = feats[:, i*512:(i+1)*512]
            reduced = self.pca_models[view].transform(view_data)
            reduced_views.append(reduced)
        final_feats = np.concatenate(reduced_views, axis=1)

        feat_dict = {t: final_feats[i] for i,t in enumerate(timesteps)}

        # state npz 합치기
        state_dict = {}
        for fname in os.listdir(traj_dir):
            if fname.endswith(".npz") and (fname.startswith("state_") or fname.startswith("states_")):
                npz_path = os.path.join(traj_dir,fname)
                try:
                    ts = int(fname.split("_")[-1].replace(".npz",""))
                except ValueError:
                    print(f"  ⚠️ Cannot parse timestep from {fname}")
                    continue

                data = np.load(npz_path)
                state_dict[ts] = {k: data[k] for k in data}

        # 저장
        with open(os.path.join(img_out,"features.pkl"),"wb") as f:
            pickle.dump(feat_dict,f)

        if state_dict:
            np.savez_compressed(os.path.join(state_out,"state.npz"),
                                **{f"t{t}": state_dict[t] for t in timesteps})
        else:
            print("  ⚠️ No robot state npz files found for this trajectory.")

        feature_array = np.array(list(feat_dict.values()))
        print(f"  저장 완료: {len(feature_array)} timesteps, feature dim {feature_array.shape[1]}")

        print(f"Image features saved:")
        print(f"  Features: {img_out}/features.pkl")
        print(f"  Total timesteps: {len(feature_array)}")
        print(f"  Feature dimension: {feature_array.shape[1]}")

def process_all(root_dir):
    config = MultiViewConfig()
    proc = MultiViewProcessor(config)
    for traj in os.listdir(root_dir):
        traj_dir = os.path.join(root_dir,traj)
        if os.path.isdir(traj_dir):
            proc.process_traj(traj_dir)

if __name__ == "__main__":
    process_all("/AILAB-summer-school-2025/failure_case/failcase2")